# Scoring your trained model

In the cell below, please load your model into `model`. Also if you used an image size for your input images that *isn't* 224x224, you'll need to set `image_size` to the size you used. The scoring code assumes square input images.

For example, this is how I loaded in my checkpoint:

```python
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import models

class FFClassifier(nn.Module):
    
    def __init__(self, in_features, hidden_features, 
                       out_features, drop_prob=0.1):
        super().__init__()
        
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(p=drop_prob)
        
    def forward(self, x):
        x = self.drop(F.relu(self.fc1(x)))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

    
def load_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    
    model = models.vgg16(pretrained=False)
    for param in model.parameters():
        param.requires_grad = False

    # Put the classifier on the pretrained network
    model.classifier = FFClassifier(25088, checkpoint['hidden'], 102)
    
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

model = load_checkpoint('/home/workspace/classifier.pt')
```

Your exact code here will depend on how you defined your network in the project. Make sure you use the absolute path to your checkpoint which should have been uploaded to the `/home/workspace` directory.

Run the cell, then after loading the data, press "Test Code" below. This can take a few minutes or more depending on the size of your network. Your model needs  to reach **at least 20% accuracy** on the test set to be recorded.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
import numpy as np
from torchvision import datasets, transforms, models
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim import lr_scheduler
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import json
import random
import os
import copy
import seaborn as sns
from PIL import Image

classifier = nn.Sequential(nn.Linear(25088, 4096), 
                           nn.ReLU(),
                           nn.Dropout(p=0.1),
                           nn.Linear(4096, 512),
                           nn.ReLU(),
                           nn.Dropout(p=0.1),
                           nn.Linear(512, 102),                           
                           nn.LogSoftmax(dim=1))

def load_model(checkpoint_path):
    '''loads the checkpoint of a model and 
       updates the classifier again accordingly'''
    checkpoint = torch.load(checkpoint_path)
    
    model = models.vgg19(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
            
    model.class_to_idx = checkpoint['class_to_idx']
    
    classifier = nn.Sequential(nn.Linear(25088, 4096), 
                           nn.ReLU(),
                           nn.Dropout(p=0.1),
                           nn.Linear(4096, 512),
                           nn.ReLU(),
                           nn.Dropout(p=0.1),
                           nn.Linear(512, 102),                           
                           nn.LogSoftmax(dim=1))
    
    model.classifier = classifier
    
    model.load_state_dict(checkpoint['model_state_dict'])
    
    return model

# Load your model to this variable
model = load_model('/home/workspace/vgg19_classifier.pth')
   
# If you used something other than 224x224 cropped images, set the correct size here
image_size = 224
# Values you used for normalizing the images. Default here are for 
# pretrained models from torchvision.
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.torch/models/vgg19-dcbb9e9d.pth
100%|██████████| 574673361/574673361 [00:04<00:00, 125237534.49it/s]
